In [1]:
from natsort import natsorted
import os
import re
from glob import glob
import json
import numpy as np
import matplotlib.pyplot as plt
import seaborn
import networkx as nx


import imageio
import cv2
import skimage
from skimage import img_as_float32, img_as_ubyte, img_as_uint
from skimage.feature import canny
from skimage.color import rgb2gray, rgb2hsv, gray2rgb, rgba2rgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from functools import partial
from tqdm.notebook import tqdm

import tensorflow as tf
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import VGG16
from tensorflow.keras import metrics

# caching with sane defaults
from cachier import cachier

In [2]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
tqdm = partial(tqdm, position=0, leave=True)

In [3]:
# Helper plot functions

def plot_img(img):
    plt.imshow(img, cmap='gray')
    plt.show

def plot_all_data(X, y):
    assert X.shape[0] == y.shape[0]
    print(X.shape[0])
    for i in range(X.shape[0]):
        fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 10))
        ax = axes.flatten()
        ax[0].imshow(X[i])
        ax[0].axis("off")
        ax[0].title.set_text(f'{i%(X.shape[0]/3)}')

        ax[1].imshow(y[i], cmap="gray")
        ax[1].axis("off")
        ax[1].title.set_text(f'{i%(X.shape[0]/3)}')
        plt.tight_layout()
        plt.show()

# UNET with VGG16 ImageNet Weights

In [4]:
class Unet:
    
    def __init__(self, input_size):
        self.input_size = input_size
        self.model = self.create_unet_vgg16()
        self.history = None
        
    def conv_block(self, x, num_filters):
        x = Conv2D(num_filters, 3, padding="same")(x)
        x = BatchNormalization()(x)
        x = Activation("relu")(x)

        x = Conv2D(num_filters, 3, padding="same")(x)
        x = BatchNormalization()(x)
        x = Activation("relu")(x)
        return x
    
    def decoder_block(self, x, skip_features, num_filters):
        x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(x)
        x = Concatenate()([x, skip_features])
        x = self.conv_block(x, num_filters)
        return x

    
    def create_unet_vgg16(self):
        """ Input """
        inputs = Input(self.input_size)

        """ Pre-trained VGG16 Model """
        vgg16 = VGG16(include_top=False, weights="imagenet", input_tensor=inputs)

        """ Encoder """
        s1 = vgg16.get_layer("block1_conv2").output         ## (512 x 512)
        s2 = vgg16.get_layer("block2_conv2").output         ## (256 x 256)
        s3 = vgg16.get_layer("block3_conv3").output         ## (128 x 128)
        s4 = vgg16.get_layer("block4_conv3").output         ## (64 x 64)

        """ Bridge """
        b1 = vgg16.get_layer("block5_conv3").output         ## (32 x 32)

        """ Decoder """
        d1 = self.decoder_block(b1, s4, 512)                     ## (64 x 64)
        d2 = self.decoder_block(d1, s3, 256)                     ## (128 x 128)
        d3 = self.decoder_block(d2, s2, 128)                     ## (256 x 256)
        d4 = self.decoder_block(d3, s1, 64)                      ## (512 x 512)

        """ Output """
        outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(d4)

        return Model(inputs, outputs, name="VGG16_U-Net")
        
    
    def train(self, X_train, X_val, y_train, y_val, lr=1e-3, batch_size=8, epochs=100):
        self.model.compile(optimizer=Adam(learning_rate=lr), loss='binary_crossentropy', metrics=['accuracy'])
        self.history = self.model.fit(X_train, y_train, 
                    batch_size=batch_size, 
                    verbose=1, 
                    epochs=epochs, 
                    validation_data=(X_test, y_test), 
                    shuffle=False)
    
    
    def plot_loss(self):
        loss = self.history.history['loss']
        val_loss = self.history.history['val_loss']
        epochs = range(1, len(loss) + 1)
        plt.plot(epochs, loss, 'y', label='Training loss')
        plt.plot(epochs, val_loss, 'r', label='Validation loss')
        plt.title('Training and validation loss')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.legend()
        plt.show()

        
    def plot_accuracy(self):
        acc = self.history.history['accuracy']
        val_acc = self.history.history['val_accuracy']
        epochs = range(1, len(acc) + 1)
        plt.plot(epochs, acc, 'y', label='Training acc')
        plt.plot(epochs, val_acc, 'r', label='Validation acc')
        plt.title('Training and validation accuracy')
        plt.xlabel('Epochs')
        plt.ylabel('Accuracy')
        plt.legend()
        plt.show()

    def predict_batch(self, x):
        y = self.model.predict(x)
        y = (y[:,:,:,0] > 0.5).astype(np.uint8)
        return y
    
    def evaluate_model(self, X, y_true):
        y_pred = self.predict_batch(X)
        y_pred = np.squeeze(y_pred)
        y_true = np.squeeze(y_true)
        print(f"Accuracy: {pixel_accuracy(y_true, y_pred)}")
        print(f"Dice: {dice(y_true, y_pred)}")
        print(f"IoU: {iou(y_true, y_pred)}")

## Evaluation Metrics

In [5]:
def dice(y_true, y_pred):
    y_true = np.asarray(y_true).astype(bool)
    y_pred = np.asarray(y_pred).astype(bool)
    
    im_sum = y_true.sum() + y_pred.sum()

    # Compute Dice coefficient
    intersection = np.logical_and(y_true, y_pred)

    return 2. * intersection.sum() / im_sum

def pixel_accuracy(y_true, y_pred):
    y_true = np.asarray(y_true).astype(np.uint8).flatten()
    y_pred = np.asarray(y_pred).astype(np.uint8).flatten()
    return accuracy_score(y_true, y_pred)


def iou(y_true, y_pred):
    y_true = np.asarray(y_true).astype(bool)
    y_pred = np.asarray(y_pred).astype(bool)
    intersection = np.logical_and(y_true, y_pred)
    union = np.logical_or(y_true, y_pred)
    iou_score = np.sum(intersection) / np.sum(union)
    return iou_score

# Read in Data

In [6]:
SIZE = (768, 1024)

DATA_PATH_PAIRS = list(
    zip(
        natsorted(
            glob(
                f"puzzle_corners_{SIZE[1]}x{SIZE[0]}/images-{SIZE[1]}x{SIZE[0]}/*.png"
            )
        ),
        natsorted(
            glob(
                f"puzzle_corners_{SIZE[1]}x{SIZE[0]}/masks-{SIZE[1]}x{SIZE[0]}/*.png"
            )
        ),
    )
)
DATA_IMGS = np.array(
    [
        img_as_float32(imageio.imread(img_path))
        for img_path, _ in tqdm(DATA_PATH_PAIRS, "Loading Images")
    ]
)
DATA_MSKS = np.array(
    [
        img_as_float32(imageio.imread(msk_path))
        for _, msk_path in tqdm(DATA_PATH_PAIRS, "Loading Masks")
    ]
)

SCALE = 0.25
MATCH_IMGS = np.array([
    cv2.resize(img, None, fx=SCALE, fy=SCALE)
    for img in tqdm(DATA_IMGS, "Resizing Images")
])
MATCH_MSKS = np.array([
    np.expand_dims(cv2.resize(img, None, fx=SCALE, fy=SCALE), axis=2)
    for img in tqdm(DATA_MSKS, "Resizing Masks")
])
    
    # Should I be normalizing the data? 
    

Loading Images:   0%|          | 0/48 [00:00<?, ?it/s]

Loading Masks:   0%|          | 0/48 [00:00<?, ?it/s]

Resizing Images:   0%|          | 0/48 [00:00<?, ?it/s]

Resizing Masks:   0%|          | 0/48 [00:00<?, ?it/s]

## Split data

In [7]:
def train_val_test_split(X, y, train_count=34, val_count=7, test_count=7, random_state=42):
    """ Split data into train, validation and test sets"""
    # Obtain percentage split values
    total_data_size = X.shape[0]
    train_size = train_count / total_data_size
    val_size = val_count / total_data_size
    test_size = val_count / total_data_size
   
    X_train, X_rem, y_train, y_rem = train_test_split(X, y, train_size=train_size, random_state=random_state)
    val_size = val_size/(val_size+test_size)
    X_val, X_test, y_val, y_test = train_test_split(X_rem, y_rem, train_size=val_size, random_state=random_state)
    
    print(f"Training images: {X_train.shape[0]}")
    print(f"Validation images: {X_val.shape[0]}")
    print(f"Test images: {X_test.shape[0]}")
    
    return X_train, X_val, X_test, y_train, y_val, y_test

In [8]:
X_train, X_val, X_test, y_train, y_val, y_test = train_val_test_split(MATCH_IMGS, MATCH_MSKS)

Training images: 34
Validation images: 7
Test images: 7


# Data augmentations

In [9]:
def apply_image_aug(aug_func, X_train, X_val, y_train, y_val, aug_target=True, args={}):
    
    X_train = np.append(X_train, aug_func(X_train[:34], **args).numpy(), 0)
    
    
    X_val = np.append(X_val, aug_func(X_val[:7], **args).numpy(), 0)
    
    
    if aug_target:
        y_train = np.append(y_train, aug_func(y_train[:34], **args).numpy(), 0)
        y_val = np.append(y_val, aug_func(y_val[:7], **args).numpy(), 0)
    else:
        y_train = np.append(y_train, y_train[:34], 0)
        y_val = np.append(y_val, y_val[:7], 0)

    return X_train, X_val, y_train, y_val

def add_data_augmentations(X_train, X_val, y_train, y_val):
    # Spatial augmentations
    
    ## Flipping
    X_train, X_val, y_train, y_val = apply_image_aug(tf.image.flip_left_right, X_train, X_val, y_train, y_val)
    X_train, X_val, y_train, y_val = apply_image_aug(tf.image.flip_up_down, X_train, X_val, y_train, y_val)    
    
    # ## Rotation
    # X_train, X_val, y_train, y_val = apply_image_aug(tf.image.rot90, X_train, X_val, y_train, y_val)
    
    # Pixel augmentations
    
    ## Brightness
    X_train, X_val, y_train, y_val = apply_image_aug(tf.image.random_brightness, X_train, X_val, y_train, y_val, aug_target=False, args={"max_delta": 0.25})    
    
    ## Contrast
    
    ## Saturation
    
    ## Hue
    
    
    return X_train, X_val, y_train, y_val

# Training model with no data Augmentation

In [10]:
unet = Unet(input_size=X_train[0].shape)

In [26]:
def objective(trial):
    
    model = Unet(input_size=X_train[0].shape).model
    # model.compile(optimizer=Adam(learning_rate=trial.suggest_categorical('lr', [1e-2, 1e-3]), loss='binary_crossentropy', metrics=['accuracy']))
    model.compile(optimizer=Adam(learning_rate=1e-3), loss='binary_crossentropy', metrics=['accuracy'])
    model.fit(X_train, y_train, 
            batch_size=trial.suggest_categorical('batch_size', [8, 16]), 
            verbose=1, 
            epochs=trial.suggest_categorical('epochs', [8, 16]), 
            validation_data=(X_test, y_test), 
            shuffle=False)
    
    y_pred = model.predict(X_val)
    y_pred = (y_pred[:,:,:,0] > 0.5).astype(np.uint8)
    
    return pixel_accuracy(y_val, y_pred)
    


In [31]:
study = optuna.create_study(direction='maximize') 

In [32]:
study.optimize(objective, n_trials=4)

Epoch 1/8
5/5 [==============================] - 2s 158ms/step - loss: 0.4386 - accuracy: 0.8027 - val_loss: 58.3468 - val_accuracy: 0.2552
Epoch 2/8
5/5 [==============================] - 1s 112ms/step - loss: 0.1269 - accuracy: 0.9706 - val_loss: 1102.4135 - val_accuracy: 0.2552
Epoch 3/8
5/5 [==============================] - 1s 112ms/step - loss: 0.0914 - accuracy: 0.9789 - val_loss: 772.3591 - val_accuracy: 0.2552
Epoch 4/8
5/5 [==============================] - 1s 113ms/step - loss: 0.0801 - accuracy: 0.9802 - val_loss: 176.9095 - val_accuracy: 0.2552
Epoch 5/8
5/5 [==============================] - 1s 112ms/step - loss: 0.0646 - accuracy: 0.9844 - val_loss: 566.5128 - val_accuracy: 0.2552
Epoch 6/8
5/5 [==============================] - 1s 112ms/step - loss: 0.0556 - accuracy: 0.9871 - val_loss: 351.6550 - val_accuracy: 0.2552
Epoch 7/8
5/5 [==============================] - 1s 112ms/step - loss: 0.0471 - accuracy: 0.9889 - val_loss: 217.5869 - val_accuracy: 0.2553
Epoch 8/8
5/5

[I 2021-11-11 17:14:05,116] Finished trial#0 resulted in value: 0.24461437406994047. Current best value is 0.24461437406994047 with parameters: {'batch_size': 8, 'epochs': 8}.


Epoch 1/16
5/5 [==============================] - 2s 157ms/step - loss: 0.3805 - accuracy: 0.8844 - val_loss: 45.9912 - val_accuracy: 0.2552
Epoch 2/16
5/5 [==============================] - 1s 112ms/step - loss: 0.1220 - accuracy: 0.9696 - val_loss: 55.1615 - val_accuracy: 0.2552
Epoch 3/16
5/5 [==============================] - 1s 112ms/step - loss: 0.0927 - accuracy: 0.9789 - val_loss: 411.6328 - val_accuracy: 0.2552
Epoch 4/16
5/5 [==============================] - 1s 114ms/step - loss: 0.0763 - accuracy: 0.9829 - val_loss: 83.0115 - val_accuracy: 0.2553
Epoch 5/16
5/5 [==============================] - 1s 113ms/step - loss: 0.0656 - accuracy: 0.9846 - val_loss: 27.4462 - val_accuracy: 0.2572
Epoch 6/16
5/5 [==============================] - 1s 112ms/step - loss: 0.0536 - accuracy: 0.9878 - val_loss: 16.4431 - val_accuracy: 0.2680
Epoch 7/16
5/5 [==============================] - 1s 116ms/step - loss: 0.0454 - accuracy: 0.9905 - val_loss: 8.1509 - val_accuracy: 0.2810
Epoch 8/16
5/

[I 2021-11-11 17:14:16,013] Finished trial#1 resulted in value: 0.30875941685267855. Current best value is 0.30875941685267855 with parameters: {'batch_size': 8, 'epochs': 16}.


Epoch 1/8
3/3 [==============================] - 2s 233ms/step - loss: 0.6021 - accuracy: 0.7021 - val_loss: 23.5327 - val_accuracy: 0.2552
Epoch 2/8
3/3 [==============================] - 0s 145ms/step - loss: 0.1837 - accuracy: 0.9589 - val_loss: 135.9676 - val_accuracy: 0.2553
Epoch 3/8
3/3 [==============================] - 0s 145ms/step - loss: 0.1138 - accuracy: 0.9732 - val_loss: 553.3302 - val_accuracy: 0.2552
Epoch 4/8
3/3 [==============================] - 0s 143ms/step - loss: 0.0921 - accuracy: 0.9791 - val_loss: 463.5516 - val_accuracy: 0.2552
Epoch 5/8
3/3 [==============================] - 1s 144ms/step - loss: 0.0779 - accuracy: 0.9811 - val_loss: 151.0225 - val_accuracy: 0.2553
Epoch 6/8
3/3 [==============================] - 0s 143ms/step - loss: 0.0676 - accuracy: 0.9835 - val_loss: 91.7852 - val_accuracy: 0.2555
Epoch 7/8
3/3 [==============================] - 0s 143ms/step - loss: 0.0565 - accuracy: 0.9874 - val_loss: 117.6369 - val_accuracy: 0.2586
Epoch 8/8
3/3 [

[I 2021-11-11 17:14:21,688] Finished trial#2 resulted in value: 0.24659075055803573. Current best value is 0.30875941685267855 with parameters: {'batch_size': 8, 'epochs': 16}.


Epoch 1/8
5/5 [==============================] - 2s 158ms/step - loss: 0.3517 - accuracy: 0.8766 - val_loss: 34.7470 - val_accuracy: 0.2552
Epoch 2/8
5/5 [==============================] - 1s 112ms/step - loss: 0.1350 - accuracy: 0.9649 - val_loss: 106.7651 - val_accuracy: 0.2552
Epoch 3/8
5/5 [==============================] - 1s 113ms/step - loss: 0.0908 - accuracy: 0.9809 - val_loss: 558.4820 - val_accuracy: 0.2552
Epoch 4/8
5/5 [==============================] - 1s 113ms/step - loss: 0.0748 - accuracy: 0.9834 - val_loss: 574.9440 - val_accuracy: 0.2552
Epoch 5/8
5/5 [==============================] - 1s 112ms/step - loss: 0.0604 - accuracy: 0.9874 - val_loss: 287.4830 - val_accuracy: 0.2552
Epoch 6/8
5/5 [==============================] - 1s 113ms/step - loss: 0.0507 - accuracy: 0.9886 - val_loss: 621.4589 - val_accuracy: 0.2552
Epoch 7/8
5/5 [==============================] - 1s 112ms/step - loss: 0.0429 - accuracy: 0.9911 - val_loss: 302.2166 - val_accuracy: 0.2554
Epoch 8/8
5/5 

[I 2021-11-11 17:14:27,966] Finished trial#3 resulted in value: 0.24707903180803573. Current best value is 0.30875941685267855 with parameters: {'batch_size': 8, 'epochs': 16}.


In [33]:
print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

Number of finished trials: 4
Best trial:
  Value: 0.30875941685267855
  Params: 
    batch_size: 8
    epochs: 16


In [25]:
print(study.best_trial.value)

0.2768467494419643


In [11]:
import optuna 
from optkeras.optkeras import OptKeras
import optkeras

study_name = "Optuna Test"

""" Step 1. Instantiate OptKeras class
You can specify arguments for Optuna's create_study method and other arguments 
for OptKeras such as enable_pruning. 
"""

ok = OptKeras(study_name=study_name,
              monitor='val_acc',
              direction='maximize')


""" Step 2. Define objective function for Optuna """

def objective(trial):
    
    """ Step 2.1. Define parameters to try using methods of optuna.trial such as 
    suggest_categorical. In this simple demo, try 2*2*2*2 = 16 parameter sets: 
    2 values specified in list for each of 4 parameters 
    (filters, kernel_size, strides, and activation for convolution).
    """
    
    model = Unet(input_size=X_train[0].shape).model
    # model.compile(optimizer=Adam(learning_rate=trial.suggest_categorical('lr', [1e-2, 1e-3]), loss='binary_crossentropy', metrics=['accuracy']))
    model.compile(optimizer=Adam(learning_rate=1e-3), loss='binary_crossentropy', metrics=['accuracy'])
    model.fit(X_train, y_train, 
            batch_size=trial.suggest_categorical('batch_size', [8, 16]), 
            verbose=1, 
            epochs=trial.suggest_categorical('epochs', [8, 16]), 
            validation_data=(X_test, y_test), 
            shuffle=False)
    
    """ Step 2.3. Return trial_best_value (or latest_value) """
    # print(ok)
    return ok.trial_best_value

""" Step 3. Run optimize. 
Set n_trials and/or timeout (in sec) for optimization by Optuna
"""
ok.optimize(objective, n_trials = 2) # Run for 3 minutes for demo


[2021-11-11 17:04:33.461733] [OptKeras] Ready for optimization. (message printed as verbose is set to 1+)
Epoch 1/16
3/3 [==============================] - 9s 1s/step - loss: 0.5843 - accuracy: 0.7060 - val_loss: 8.2917 - val_accuracy: 0.2565
Epoch 2/16
3/3 [==============================] - 0s 146ms/step - loss: 0.1972 - accuracy: 0.9577 - val_loss: 147.7888 - val_accuracy: 0.2552
Epoch 3/16
3/3 [==============================] - 0s 145ms/step - loss: 0.1335 - accuracy: 0.9697 - val_loss: 311.5243 - val_accuracy: 0.2552
Epoch 4/16
3/3 [==============================] - 1s 159ms/step - loss: 0.0994 - accuracy: 0.9793 - val_loss: 851.0480 - val_accuracy: 0.2552
Epoch 5/16
3/3 [==============================] - 1s 158ms/step - loss: 0.0905 - accuracy: 0.9803 - val_loss: 250.8889 - val_accuracy: 0.2552
Epoch 6/16
3/3 [==============================] - 1s 161ms/step - loss: 0.0758 - accuracy: 0.9853 - val_loss: 132.1783 - val_accuracy: 0.2552
Epoch 7/16
3/3 [==============================]

[I 2021-11-11 17:04:50,962] Finished trial#0 resulted in value: inf. Current best value is inf with parameters: {'batch_size': 16, 'epochs': 16}.


Epoch 1/16
5/5 [==============================] - 4s 171ms/step - loss: 0.3047 - accuracy: 0.9084 - val_loss: 12.6150 - val_accuracy: 0.2552
Epoch 2/16
5/5 [==============================] - 1s 121ms/step - loss: 0.1080 - accuracy: 0.9712 - val_loss: 69.1161 - val_accuracy: 0.2552
Epoch 3/16
5/5 [==============================] - 1s 121ms/step - loss: 0.0853 - accuracy: 0.9789 - val_loss: 238.0980 - val_accuracy: 0.2552
Epoch 4/16
5/5 [==============================] - 1s 122ms/step - loss: 0.0679 - accuracy: 0.9835 - val_loss: 234.3112 - val_accuracy: 0.2552
Epoch 5/16
5/5 [==============================] - 1s 122ms/step - loss: 0.0547 - accuracy: 0.9869 - val_loss: 237.6750 - val_accuracy: 0.2553
Epoch 6/16
5/5 [==============================] - 1s 123ms/step - loss: 0.0442 - accuracy: 0.9893 - val_loss: 124.5253 - val_accuracy: 0.2553
Epoch 7/16
5/5 [==============================] - 1s 123ms/step - loss: 0.0362 - accuracy: 0.9910 - val_loss: 27.1986 - val_accuracy: 0.2563
Epoch 8/1

[I 2021-11-11 17:05:04,391] Finished trial#1 resulted in value: inf. Current best value is inf with parameters: {'batch_size': 16, 'epochs': 16}.


[2021-11-11 17:05:04.395357] Trial#: 1, value: inf| Best trial#: 0, value: inf, params: {'batch_size': 16, 'epochs': 16}


In [12]:
pip install optkeras==0.0.7

Note: you may need to restart the kernel to use updated packages.


In [13]:
print(asdfsdf)

NameError: name 'asdfsdf' is not defined

In [ ]:
unet.train(X_train, X_val, y_train, y_val, lr=1e-3, batch_size=16, epochs=100)

In [ ]:
unet.plot_loss()

In [ ]:
unet.plot_accuracy()

In [ ]:
unet.evaluate_model(X_test, y_test)

In [ ]:
unet.evaluate_model(X_train, y_train)

In [ ]:
unet.evaluate_model(X_val, y_val)

# Training model with data Augmentation

In [ ]:
X_train, X_val, y_train, y_val = add_data_augmentations(X_train, X_val, y_train, y_val)
print(f"Training images: {X_train.shape[0]}")
print(f"Validation images: {X_val.shape[0]}")
print(f"Test images: {X_test.shape[0]}")

In [ ]:
unet = Unet(input_size=X_train[0].shape)

In [ ]:
unet.train(X_train, X_val, y_train, y_val, lr=1e-3, batch_size=16, epochs=100)

In [ ]:
unet.plot_loss()

In [ ]:
unet.plot_accuracy()

In [ ]:
unet.evaluate_model(X_test, y_test)

In [ ]:
unet.evaluate_model(X_train, y_train)

In [ ]:
unet.evaluate_model(X_val, y_val)

In [ ]:
from tensorflow.keras.metrics import MeanIoU
n_classes = 2
IOU_keras = MeanIoU(num_classes=n_classes)  
IOU_keras.update_state(unet.predict_batch(X_test), y_test)
print("Mean IoU =", IOU_keras.result().numpy())

In [ ]:
from tensorflow.keras.metrics import IoU

In [ ]:
from tensorflow.keras.metrics import Accuracy
acc_keras = Accuracy()  
acc_keras.update_state(unet.predict_batch(X_test), y_test)
print("Acc =", IOU_keras.result().numpy())

In [ ]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

In [ ]:
temp = KerasClassifier(unet.model)

In [ ]:
import keras_tuner as kt

In [ ]:
unet = Unet(input_size=X_train[0].shape)